In [100]:
from DFTDataFrame.Tools import (
    create_frame,
    get_pathtofile,
    read_strucfile,
    fill_struc_gap,
    get_project_root,
    get_zpe_entropies,
    Frequency,
    getVacuum,
    Imaginary,
    converged,
    notconverged,
)

from os import path as ospath
from ase import Atoms


# from DFTDataFrame.Tools import lines_that_start_with

# from DFTDataFrame.Tools import get_zpe
from DFTDataFrame.Tools import get_entropies
from DFTDataFrame.Tools import Atommultiindex
from DFTDataFrame.Tools import gas_free_G
from DFTDataFrame.Tools import ads_free_G

from pandas import DataFrame
from numpy import NaN
import pandas as pd
from numpy import round
import logging

In [101]:
import warnings
warnings.simplefilter('ignore')

In [102]:
# Creating a dataframe from a given directory and its subdirectories.
# The Flag file is the file inidicating in which folders it should look for. 
# Calc_file is read by ase.read to an Atoms object and reading all information from.

root=get_project_root()
Cu_slabs = create_frame('data/Cu/slabs/', calc_file='final.traj', flag_file='final.traj', droplist=['clean']) #Droplist contains strings which can be ommited when creating the Name/Index
Cu_bulk = create_frame('data/Cu/bulk/', calc_file='final.traj', flag_file='final.traj')
Gas = create_frame('data/Gas/', calc_file='final.traj', flag_file='final.traj')


CRITICAL:root:data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms/final.traj energy could not be read


Found 8 folders in data/Cu/slabs/
Found 1 folders in data/Cu/bulk/
Found 3 folders in data/Gas/


In [103]:
# The DataFrame has 12 columns
# Path: path to the folder, 
# Files: files in the folder, usefule to check if ACF.dat or DOSCAR is present
# the Name: name which is shorter and easier to remember than the Path,
# E: the final energy
# struc: the atoms object of the final structure.
# fmax: the forces
# timestamp: the last modified date, used to update only what is necessary the frame 
# a,b,c, gamma: a,b,c and gamma of the unit cell
# Formula: the chemical formula of the structures
Cu_slabs

,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula
100-2x2x4,data/Cu/slabs/100/clean/2x2x4,100-2x2x4,"[final.traj, CO, ACF.dat, CONTCAR, O, H, OH, H...",-5.510428,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0081,1.710790e+09,5.182523,5.182523,21.496896,90.0,Cu16
100-2x2x4-CO-1,data/Cu/slabs/100/clean/2x2x4/CO/1,100-2x2x4-CO-1,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-18.173690,"(Atom('Cu', [1.8956307871401654, 1.99563078714...",0.0064,1.710789e+09,5.182523,5.182523,21.496896,90.0,CCu16O
100-2x2x4-O,data/Cu/slabs/100/clean/2x2x4/O,100-2x2x4-O,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-10.682109,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0069,1.706996e+09,5.182523,5.182523,21.496896,90.0,Cu16O
100-2x2x4-H,data/Cu/slabs/100/clean/2x2x4/H,100-2x2x4-H,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-8.968309,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0073,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16
100-2x2x4-OH-complete,data/Cu/slabs/100/clean/2x2x4/OH/complete,100-2x2x4-OH-complete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16O
100-2x2x4-OH-incomplete,data/Cu/slabs/100/clean/2x2x4/OH/incomplete,100-2x2x4-OH-incomplete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698505e+09,5.182523,5.182523,21.496896,90.0,HCu16O
100-2x2x4-OH-missmatch,data/Cu/slabs/100/clean/2x2x4/OH/missmatch,100-2x2x4-OH-missmatch,"[final.traj, CONTCAR]",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698506e+09,5.182523,5.182523,21.496896,90.0,HCu16O
100-2x2x4-OH-zeroAtoms,data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms,100-2x2x4-OH-zeroAtoms,"[out.txt, final.traj, CONTCAR]",0.000000,(),0.0000,1.698506e+09,0.000000,0.000000,0.000000,0.0,0


There is a column which is called "struc" which should contain the structure from which we derived the forces and the enrgy.

It can be that some calculations are missing that, because they don't have a "final.traj" file. 
For example when a calculation is still running or has Failed at some pint. 

To still get a structure, energy and the forces you can fill those empty columns from another file with the following command:

In [104]:
Cu_slabs['struc'], Cu_slabs['E'], Cu_slabs['fmax'] = zip(*Cu_slabs.apply(fill_struc_gap, args=['OUTCAR'], axis=1))
Cu_slabs#In our case nothing changed

,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula
100-2x2x4,data/Cu/slabs/100/clean/2x2x4,100-2x2x4,"[final.traj, CO, ACF.dat, CONTCAR, O, H, OH, H...",-5.510428,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0081,1.710790e+09,5.182523,5.182523,21.496896,90.0,Cu16
100-2x2x4-CO-1,data/Cu/slabs/100/clean/2x2x4/CO/1,100-2x2x4-CO-1,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-18.173690,"(Atom('Cu', [1.8956307871401654, 1.99563078714...",0.0064,1.710789e+09,5.182523,5.182523,21.496896,90.0,CCu16O
100-2x2x4-O,data/Cu/slabs/100/clean/2x2x4/O,100-2x2x4-O,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-10.682109,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0069,1.706996e+09,5.182523,5.182523,21.496896,90.0,Cu16O
100-2x2x4-H,data/Cu/slabs/100/clean/2x2x4/H,100-2x2x4-H,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-8.968309,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0073,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16
100-2x2x4-OH-complete,data/Cu/slabs/100/clean/2x2x4/OH/complete,100-2x2x4-OH-complete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16O
100-2x2x4-OH-incomplete,data/Cu/slabs/100/clean/2x2x4/OH/incomplete,100-2x2x4-OH-incomplete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698505e+09,5.182523,5.182523,21.496896,90.0,HCu16O
100-2x2x4-OH-missmatch,data/Cu/slabs/100/clean/2x2x4/OH/missmatch,100-2x2x4-OH-missmatch,"[final.traj, CONTCAR]",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698506e+09,5.182523,5.182523,21.496896,90.0,HCu16O
100-2x2x4-OH-zeroAtoms,data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms,100-2x2x4-OH-zeroAtoms,"[out.txt, final.traj, CONTCAR]",0.000000,(),0.0000,1.698506e+09,0.000000,0.000000,0.000000,0.0,0


In [105]:
#Sometimes we want the structure from specific file, e.g the CONTCAR file for the Bader charge
Cu_slabs['CONTCAR'] = Cu_slabs.apply(read_strucfile,args=['CONTCAR'], axis=1)

CRITICAL:root:CONTCAR empty in 100-2x2x4-OH-zeroAtoms


# A few tests

In [106]:
Cu_slabs[Cu_slabs.apply(lambda x: len(x.struc), axis=1) == 0] # shows rows which have 0 atoms


,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula,CONTCAR
100-2x2x4-OH-zeroAtoms,data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms,100-2x2x4-OH-zeroAtoms,"[out.txt, final.traj, CONTCAR]",0.0,(),0.0,1.698506e+09,0.0,0.0,0.0,0.0,0,()


In [107]:
notconverged(Cu_slabs, convergence_threshold=0.01) # shows Calculations which are not converged (force) or have forces exactly 0

,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula,CONTCAR
100-2x2x4-OH-zeroAtoms,data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms,100-2x2x4-OH-zeroAtoms,"[out.txt, final.traj, CONTCAR]",0.0,(),0.0,1.698506e+09,0.0,0.0,0.0,0.0,0,()


In [108]:
converged(Cu_slabs) #gives you only the force converged calculations

,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula,CONTCAR
100-2x2x4,data/Cu/slabs/100/clean/2x2x4,100-2x2x4,"[final.traj, CO, ACF.dat, CONTCAR, O, H, OH, H...",-5.510428,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0081,1.710790e+09,5.182523,5.182523,21.496896,90.0,Cu16,"(Atom('Cu', [1.2956307871401649, 1.29563078714..."
100-2x2x4-CO-1,data/Cu/slabs/100/clean/2x2x4/CO/1,100-2x2x4-CO-1,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-18.173690,"(Atom('Cu', [1.8956307871401654, 1.99563078714...",0.0064,1.710789e+09,5.182523,5.182523,21.496896,90.0,CCu16O,"(Atom('Cu', [1.895630787140176, 1.995630787140..."
100-2x2x4-O,data/Cu/slabs/100/clean/2x2x4/O,100-2x2x4-O,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-10.682109,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0069,1.706996e+09,5.182523,5.182523,21.496896,90.0,Cu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714..."
100-2x2x4-H,data/Cu/slabs/100/clean/2x2x4/H,100-2x2x4-H,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-8.968309,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0073,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16,"(Atom('Cu', [1.2956307871401649, 1.29563078714..."
100-2x2x4-OH-complete,data/Cu/slabs/100/clean/2x2x4/OH/complete,100-2x2x4-OH-complete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714..."
100-2x2x4-OH-incomplete,data/Cu/slabs/100/clean/2x2x4/OH/incomplete,100-2x2x4-OH-incomplete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698505e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714..."
100-2x2x4-OH-missmatch,data/Cu/slabs/100/clean/2x2x4/OH/missmatch,100-2x2x4-OH-missmatch,"[final.traj, CONTCAR]",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698506e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714..."


In [109]:

Cu_slabs['Vacuum'] = Cu_slabs.apply(getVacuum, axis=1)

#Return the Vacuum of all calculations which have a vacuum higher than 0 and smaller than 15
Cu_slabs[(Cu_slabs.Vacuum < 15) & (Cu_slabs.Vacuum > 0)].sort_values('Vacuum').Vacuum

0 atoms in: 100-2x2x4-OH-zeroAtoms


100-2x2x4-CO-1             12.842346
100-2x2x4-OH-complete      13.754283
100-2x2x4-OH-incomplete    13.754283
100-2x2x4-OH-missmatch     13.754283
100-2x2x4-H                14.915284
Name: Vacuum, dtype: float64

## Filtering

In [110]:
#Not all calculations are always relevent. For example calculations in test folders can often be ignored.
#In this set we don't have any but if you would like to create a new frame with the filtered results assign it to a variable, possibly the same:
#Cu_slabs = Cu_slabs[~Cu_slabs.Name.str.contains('test')]
Cu_slabs[Cu_slabs.Name.str.contains('test')]


,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula,CONTCAR,Vacuum


In [111]:
# The ~ is a NOT operator here. So all entries that do NOT contains test are shown:
Cu_slabs[~Cu_slabs.Name.str.contains('test')]


,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula,CONTCAR,Vacuum
100-2x2x4,data/Cu/slabs/100/clean/2x2x4,100-2x2x4,"[final.traj, CO, ACF.dat, CONTCAR, O, H, OH, H...",-5.510428,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0081,1.710790e+09,5.182523,5.182523,21.496896,90.0,Cu16,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",16.030715
100-2x2x4-CO-1,data/Cu/slabs/100/clean/2x2x4/CO/1,100-2x2x4-CO-1,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-18.173690,"(Atom('Cu', [1.8956307871401654, 1.99563078714...",0.0064,1.710789e+09,5.182523,5.182523,21.496896,90.0,CCu16O,"(Atom('Cu', [1.895630787140176, 1.995630787140...",12.842346
100-2x2x4-O,data/Cu/slabs/100/clean/2x2x4/O,100-2x2x4-O,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-10.682109,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0069,1.706996e+09,5.182523,5.182523,21.496896,90.0,Cu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",15.229739
100-2x2x4-H,data/Cu/slabs/100/clean/2x2x4/H,100-2x2x4-H,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-8.968309,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0073,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",14.915284
100-2x2x4-OH-complete,data/Cu/slabs/100/clean/2x2x4/OH/complete,100-2x2x4-OH-complete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",13.754283
100-2x2x4-OH-incomplete,data/Cu/slabs/100/clean/2x2x4/OH/incomplete,100-2x2x4-OH-incomplete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698505e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",13.754283
100-2x2x4-OH-missmatch,data/Cu/slabs/100/clean/2x2x4/OH/missmatch,100-2x2x4-OH-missmatch,"[final.traj, CONTCAR]",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698506e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",13.754283
100-2x2x4-OH-zeroAtoms,data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms,100-2x2x4-OH-zeroAtoms,"[out.txt, final.traj, CONTCAR]",0.000000,(),0.0000,1.698506e+09,0.000000,0.000000,0.000000,0.0,0,(),0.000000


## Adsorption energies

##### To make a transparent and comprehensible calculation of adsorption energies, we add not just a columns with the adsorption energy but we add all values that are important for this step in individual columns:
1. The name and from that 
2. the energy of the reference surface
3. The name and from that
4. the energy of the adsorbate molecule

The energy can be the potential energy which is then just a float or it can be a function for the free energy with a variable of the Temperature. (e.g as sympy object). Then the final adsorption energy also becomes a function of the Temperature.

In the folder structure as it is presented in the tests directory here, reference surface can be obtained by just cutting the part after the adosrbate molecule. e.g (100-clean-2x2x4-OH  ):

Path to reference surface:  211-clean-3x3x4

Path to adsorbate:          211-clean-3x3x4-OH-AA


For the adsorbates it is a little more complex because we do not often have one single calculation that refers to the adsorbate. So we need a dictionary with the possible adsorbates and their respective energy. By creating different dictionaries references

In [112]:
Gasreferences = {
    "OH": Gas.loc["H2O"]["E"] - Gas.loc["H2"]["E"] / 2,
    "CO": Gas.loc["CO"]["E"],
    "H": Gas.loc["H2"]["E"] / 2,
    "O": Gas.loc["H2O"]["E"] - Gas.loc["H2"]["E"],
}


In [113]:
def get_adsorbate_and_surface_ref(row, Gasreferences):
    ''' return adsorbate and reference surface'''
    Name = row.Name
    for adsorbate in Gasreferences.keys():
        if "-" + adsorbate + "-" in Name:
            '''if adsorbate is in the Name: -OH-'''
            surface_ref = Name.split('-'+adsorbate+'-')[0]
            return adsorbate, surface_ref
        elif Name.endswith("-" + adsorbate):
            '''if Name ends with the adsorbate: -OH '''
            surface_ref = Name.split('-'+adsorbate)[0]
            return adsorbate, surface_ref
        elif "-" + adsorbate + "x" in Name:
            '''if -OHx2 the adsorbate appears twice'''
            surface_ref = Name.split('-'+adsorbate+'x')[0]
            return adsorbate, surface_ref
        else:
            continue
    logging.info('no adsorbate' + Name)
    return NaN, NaN


def add_surface_ref(Frame, Gasreferences):
    Frame['adsorbate'], Frame['surface_ref'] = zip(*Frame.apply(get_adsorbate_and_surface_ref, args=[Gasreferences], axis=1))
    return Frame

### OR ###


In [114]:
Cu_slabs = add_surface_ref(Cu_slabs, Gasreferences)
Cu_slabs[['surface_ref', 'adsorbate']]
### OR ###
Cu_slabs

,Path,Name,files,E,struc,fmax,timestamp,a,b,c,gamma,Formula,CONTCAR,Vacuum,adsorbate,surface_ref
100-2x2x4,data/Cu/slabs/100/clean/2x2x4,100-2x2x4,"[final.traj, CO, ACF.dat, CONTCAR, O, H, OH, H...",-5.510428,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0081,1.710790e+09,5.182523,5.182523,21.496896,90.0,Cu16,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",16.030715,NaN,NaN
100-2x2x4-CO-1,data/Cu/slabs/100/clean/2x2x4/CO/1,100-2x2x4-CO-1,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-18.173690,"(Atom('Cu', [1.8956307871401654, 1.99563078714...",0.0064,1.710789e+09,5.182523,5.182523,21.496896,90.0,CCu16O,"(Atom('Cu', [1.895630787140176, 1.995630787140...",12.842346,CO,100-2x2x4
100-2x2x4-O,data/Cu/slabs/100/clean/2x2x4/O,100-2x2x4-O,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-10.682109,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0069,1.706996e+09,5.182523,5.182523,21.496896,90.0,Cu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",15.229739,O,100-2x2x4
100-2x2x4-H,data/Cu/slabs/100/clean/2x2x4/H,100-2x2x4-H,"[out.txt, final.traj, ACF.dat, CONTCAR, vib.eq...",-8.968309,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0073,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",14.915284,H,100-2x2x4
100-2x2x4-OH-complete,data/Cu/slabs/100/clean/2x2x4/OH/complete,100-2x2x4-OH-complete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.706996e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",13.754283,OH,100-2x2x4
100-2x2x4-OH-incomplete,data/Cu/slabs/100/clean/2x2x4/OH/incomplete,100-2x2x4-OH-incomplete,"[vib.17y-.pckl, vib.17x+.pckl, vib.17z+.pckl, ...",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698505e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",13.754283,OH,100-2x2x4
100-2x2x4-OH-missmatch,data/Cu/slabs/100/clean/2x2x4/OH/missmatch,100-2x2x4-OH-missmatch,"[final.traj, CONTCAR]",-14.752907,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",0.0091,1.698506e+09,5.182523,5.182523,21.496896,90.0,HCu16O,"(Atom('Cu', [1.2956307871401649, 1.29563078714...",13.754283,OH,100-2x2x4
100-2x2x4-OH-zeroAtoms,data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms,100-2x2x4-OH-zeroAtoms,"[out.txt, final.traj, CONTCAR]",0.000000,(),0.0000,1.698506e+09,0.000000,0.000000,0.000000,0.0,0,(),0.000000,OH,100-2x2x4


In [115]:
def get_ads_and_ref_energies(row, Gasreferences, surface_reference, adsorbate_col='adsorbate', surface_ref_col='surface_ref'):
    ''' return adsorbate and reference surface energies'''
    ads = row[adsorbate_col]
    ref = row[surface_ref_col]
#    adsE = Gasreferences[ads].E 

    try:
        adsE = Gasreferences[ads]
    except KeyError:
        logging.error(str(ads) + 'KeyError not in Gasreferences')
        adsE = NaN
    except AttributeError:
        adsE = NaN
    except Exception:
        logging.error(str(ads) + 'Exception not in Gasreferences')
        adsE = NaN
    try:
        surfE = surface_reference.loc[ref].E
    except KeyError:
        logging.error(str(ref) + 'KeyError not in surface references')
        surfE = NaN
    except AttributeError:
        surfE = NaN
    except Exception:
        surfE = NaN
        logging.error(ref + '= not in surface references')
    return adsE, surfE


def add_adsorbate_and_ref_energies(Frame, Gasreferences, surface_references):
    Frame['adsorbate_E'], Frame['surface_ref_E'] = zip(*Frame.apply(get_ads_and_ref_energies, args=[Gasreferences, surface_references], axis=1))
    return Frame


Cu_slabs =  add_adsorbate_and_ref_energies(Cu_slabs, Gasreferences, Cu_slabs)# The Frame with the Adsorbates and the Frame with the surface references are the same in this case.
Cu_slabs['ads_E'] = Cu_slabs.E  - Cu_slabs['adsorbate_E'] - Cu_slabs['surface_ref_E']
Cu_slabs[['E', 'adsorbate_E', 'surface_ref_E', 'ads_E']]

ERROR:root:nanKeyError not in Gasreferences
ERROR:root:nanKeyError not in surface references


,E,adsorbate_E,surface_ref_E,ads_E
100-2x2x4,-5.510428,NaN,NaN,NaN
100-2x2x4-CO-1,-18.173690,-12.064823,-5.510428,-0.598439
100-2x2x4-O,-10.682109,-5.646424,-5.510428,0.474743
100-2x2x4-H,-8.968309,-3.581935,-5.510428,0.124054
100-2x2x4-OH-complete,-14.752907,-9.228359,-5.510428,-0.014120
100-2x2x4-OH-incomplete,-14.752907,-9.228359,-5.510428,-0.014120
100-2x2x4-OH-missmatch,-14.752907,-9.228359,-5.510428,-0.014120
100-2x2x4-OH-zeroAtoms,0.000000,-9.228359,-5.510428,14.738787


# Frequencies

In [116]:
#Frequency is a function that takes a DataFrame and will read the frequencies in vib.xyz and a column 'Frequencies' with the found data.
Frequency(Cu_slabs)
Cu_slabs["Frequency"]

100-2x2x4                                                            []
100-2x2x4-CO-1                 [6.7, 38.7, 241.7, 243.7, 293.3, 2020.5]
100-2x2x4-O                                       [297.1, 317.8, 319.1]
100-2x2x4-H                                     [236.5, 1134.1, 1178.7]
100-2x2x4-OH-complete      [77.8*I, 207.5, 332.8, 520.2, 583.8, 3706.3]
100-2x2x4-OH-incomplete                                              []
100-2x2x4-OH-missmatch                                               []
100-2x2x4-OH-zeroAtoms                                               []
Name: Frequency, dtype: object

In [117]:
#Imaginary is a function that looks at the Frequency columns and returns the imaginary frequencies
Imaginary(Cu_slabs)

100-2x2x4-OH-complete    [77.8*I]
Name: Imaginary, dtype: object

## Delta ads free G 

##### Instead of the pure energy we will use a sympy object which contains a mathematical funtion of the Temperature

In [118]:
Cu_slabs = get_entropies(Cu_slabs, out_file='out.txt')
Cu_slabs[[
            "E_pot",
            "E_ZPE",
            "Cv_trans",
            "Cv_rot",
            "Cv_vib",
            "C_vtoC_p",
            "S_trans",
            "S_rot",
            "S_elec",
            "S_vib",
            "Sbar",
            "S",
        ]]

data/Cu/slabs/100/clean/2x2x4 
 no file
data/Cu/slabs/100/clean/2x2x4/OH/complete 
 {}
data/Cu/slabs/100/clean/2x2x4/OH/incomplete 
 {}
data/Cu/slabs/100/clean/2x2x4/OH/missmatch 
 no file
data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms 
 {}


,E_pot,E_ZPE,Cv_trans,Cv_rot,Cv_vib,C_vtoC_p,S_trans,S_rot,S_elec,S_vib,Sbar,S
100-2x2x4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100-2x2x4-CO-1,-18.174,0.176,0.065,0.065,0.171,0.043,0.002138,0.001695,0.0,0.001045,-0.000001,0.004877
100-2x2x4-O,-10.682,0.058,0.065,0.065,0.080,0.043,0.002137,0.001687,0.0,0.000296,-0.000001,0.004118
100-2x2x4-H,-8.968,0.158,0.065,0.065,0.041,0.043,0.002135,0.001684,0.0,0.000149,-0.000001,0.003966
100-2x2x4-OH-complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100-2x2x4-OH-incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100-2x2x4-OH-missmatch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100-2x2x4-OH-zeroAtoms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
Gas = get_entropies(Gas, 'out.txt')
Gas[[
            "E_pot",
            "E_ZPE",
            "Cv_trans",
            "Cv_rot",
            "Cv_vib",
            "C_vtoC_p",
            "S_trans",
            "S_rot",
            "S_elec",
            "S_vib",
            "Sbar",
            "S",
        ]]

,E_pot,E_ZPE,Cv_trans,Cv_rot,Cv_vib,C_vtoC_p,S_trans,S_rot,S_elec,S_vib,Sbar,S
H2O,-12.810,0.575,0.065,0.065,0.002,0.043,0.001613,0.000582,0.0,0.000005,-0.000001,0.002199
H2,-7.164,0.276,0.065,0.043,0.000,0.043,0.001330,0.000175,0.0,0.000000,-0.000001,0.001504
CO,-12.065,0.132,0.065,0.043,0.001,0.043,0.001670,0.000536,0.0,0.000001,-0.000001,0.002206


In [120]:
Gas['G(T)'] = Gas.apply(gas_free_G, axis=1)


In [121]:
from sympy import symbols
Entropies = DataFrame(
    columns=[
        "E_pot",
        "E_ZPE",
        "Cv_trans",
        "Cv_rot",
        "Cv_vib",
        "C_vtoC_p",
        "S_trans",
        "S_rot",
        "S_elec",
        "S_vib",
        "Sbar",
        "S",
    ],
    index=Cu_slabs.index,
)
(
    Entropies["E_pot"],
    Entropies["E_ZPE"],
    Entropies["Cv_trans"],
    Entropies["Cv_rot"],
    Entropies["Cv_vib"],
    Entropies["C_vtoC_p"],
    Entropies["S_trans"],
    Entropies["S_rot"],
    Entropies["S_elec"],
    Entropies["S_vib"],
    Entropies["Sbar"],
    Entropies["S"],
) = zip(*Cu_slabs.apply(get_zpe_entropies, axis=1))
# Cu_slabs['zpe'], Entropies[['E_pot', 'E_ZPE', 'Cv_trans', 'Cv_rot', 'Cv_vib', 'C_vtoC_p', 'S_trans','S_rot', 'S_elec', 'S_vib', 'S']] = zip(*Cu_slabs.apply(get_zpe_entropies, axis=1))
# Cu_slabs[Cu_slabs['zpe'].isna() ].sort_values('Time')['Time'].apply(time.ctime)
try:
    Cu_slabs = Cu_slabs.join(Entropies)
except:
    Cu_slabs.update(Entropies)


Entropies = DataFrame(
    columns=[
        "E_pot",
        "E_ZPE",
        "Cv_trans",
        "Cv_rot",
        "Cv_vib",
        "C_vtoC_p",
        "S_trans",
        "S_rot",
        "S_elec",
        "S_vib",
        "Sbar",
        "S",
    ],
    index=Gas.index,
)
(
    Entropies["E_pot"],
    Entropies["E_ZPE"],
    Entropies["Cv_trans"],
    Entropies["Cv_rot"],
    Entropies["Cv_vib"],
    Entropies["C_vtoC_p"],
    Entropies["S_trans"],
    Entropies["S_rot"],
    Entropies["S_elec"],
    Entropies["S_vib"],
    Entropies["Sbar"],
    Entropies["S"],
) = zip(*Gas.apply(get_zpe_entropies, axis=1))
try:
    Gas = Gas.join(Entropies)
except:
    Gas.update(Entropies)


kb = 8.617333262145e-5
T = symbols("Temp")
x = symbols("x")






data/Cu/slabs/100/clean/2x2x4 
 no file
data/Cu/slabs/100/clean/2x2x4/OH/complete 
 {}
data/Cu/slabs/100/clean/2x2x4/OH/incomplete 
 {}
data/Cu/slabs/100/clean/2x2x4/OH/missmatch 
 no file
data/Cu/slabs/100/clean/2x2x4/OH/zeroAtoms 
 {}


In [122]:
def gas_free_G(row):
    E = row["E"]
    ZPE = row["E_ZPE"]
    Cv_trans = row["Cv_trans"]
    Cv_rot = row["Cv_rot"]
    Cv_vib = row["Cv_vib"]
    S_trans = row["S_trans"]
    S_rot = row["S_rot"]
    S_vib = row["S_vib"]
    G = (
        float(E)
        + float(ZPE)
        + Cv_trans
        + Cv_rot
        + Cv_vib
        - T * (S_trans + S_rot + S_vib)
    )
    return G

Gas["free_G"] = Gas.apply(gas_free_G, axis=1)

In [123]:

def ads_free_G(row):
    E = row["E"]
    ZPE = row["E_ZPE"]
    Cv_vib = row["Cv_vib"]
    S_vib = row["S_vib"]
    G = float(E) + float(ZPE) + Cv_vib - T * (S_vib)
    return G


Cu_slabs["free_G"] = Cu_slabs.apply(ads_free_G, axis=1)


In [124]:
Gasreferences_G = {
    "OH": Gas.loc["H2O"]["free_G"] - Gas.loc["H2"]["free_G"] / 2,
    "CO": Gas.loc["CO"]["free_G"],
    "H": Gas.loc["H2"]["free_G"] / 2,
    "O": Gas.loc["H2O"]["free_G"] - Gas.loc["H2"]["free_G"],
}

In [125]:
#Now we do the same with free_G as we did it for the E

def get_ads_and_ref_energies(row, Gasreferences, surface_reference, adsorbate_col='adsorbate', surface_ref_col='surface_ref'):
    ''' return adsorbate and reference surface energies'''
    ads = row[adsorbate_col]
    ref = row[surface_ref_col]
#    adsE = Gasreferences[ads].E 

    try:
        adsE = Gasreferences[ads]
    except KeyError:
        logging.error(str(ads) + ' KeyError not in Gasreferences')
        adsE = NaN
    except AttributeError:
        adsE = NaN
    except Exception:
        logging.error(str(ads) + ' Exception not in Gasreferences')
        adsE = NaN
    try:
        surfE = surface_reference.loc[ref].E
    except KeyError:
        logging.error(str(ref) + ' KeyError not in surface references')
        surfE = NaN
    except AttributeError:
        surfE = NaN
    except Exception:
        surfE = NaN
        logging.error(ref + '= not in surface references')
    return adsE, surfE


def add_adsorbate_and_ref_energies(Frame, Gasreferences, surface_references):
    Frame['adsorbate_G'], Frame['surface_ref_G'] = zip(*Frame.apply(get_ads_and_ref_energies, args=[Gasreferences, surface_references], axis=1))
    return Frame


Cu_slabs =  add_adsorbate_and_ref_energies(Cu_slabs, Gasreferences_G, Cu_slabs)# The Frame with the Adsorbates and the Frame with the surface references are the same in this case.
Cu_slabs['ads_G'] = Cu_slabs.free_G  - Cu_slabs['adsorbate_G'] - Cu_slabs['surface_ref_G']
Cu_slabs[['free_G', 'adsorbate_G', 'surface_ref_G', 'ads_G']]

ERROR:root:nan KeyError not in Gasreferences
ERROR:root:nan KeyError not in surface references


,free_G,adsorbate_G,surface_ref_G,ads_G
100-2x2x4,nan,NaN,NaN,nan
100-2x2x4-CO-1,-0.0010445*Temp - 17.82668953,-0.0022076*Temp - 11.8238225,-5.510428,0.0011631*Temp - 0.492438990000001
100-2x2x4-O,-0.0002956*Temp - 10.5441089,-0.0006942*Temp - 5.323424,-5.510428,0.0003986*Temp + 0.289743140000002
100-2x2x4-H,-0.0001487*Temp - 8.76930931,-0.0007528*Temp - 3.389934775,-5.510428,0.0006041*Temp + 0.131053505
100-2x2x4-OH-complete,nan,-0.001447*Temp - 8.713358775,-5.510428,nan
100-2x2x4-OH-incomplete,nan,-0.001447*Temp - 8.713358775,-5.510428,nan
100-2x2x4-OH-missmatch,nan,-0.001447*Temp - 8.713358775,-5.510428,nan
100-2x2x4-OH-zeroAtoms,nan,-0.001447*Temp - 8.713358775,-5.510428,nan


In [126]:
#The ads_G is still a function of the Temperature. To evaluate you need to get the sympy object in the column and use its function .subs() to get a singel number.  
Cu_slabs['adsG_T500'] = Cu_slabs['ads_G'].apply(lambda x: x.subs("Temp", 500))
Cu_slabs['adsG_T500']

100-2x2x4                                 nan
100-2x2x4-CO-1             0.0891110099999990
100-2x2x4-O                 0.489043140000002
100-2x2x4-H                 0.433103505000000
100-2x2x4-OH-complete                     nan
100-2x2x4-OH-incomplete                   nan
100-2x2x4-OH-missmatch                    nan
100-2x2x4-OH-zeroAtoms                    nan
Name: adsG_T500, dtype: object

In [127]:
Cu_slabs['ranked'] = Cu_slabs.groupby(["surface_ref", "adsorbate"])["ads_E"].rank(method="min", ascending=True)
Cu_slabs['ranked']


100-2x2x4                  NaN
100-2x2x4-CO-1             1.0
100-2x2x4-O                1.0
100-2x2x4-H                1.0
100-2x2x4-OH-complete      1.0
100-2x2x4-OH-incomplete    1.0
100-2x2x4-OH-missmatch     1.0
100-2x2x4-OH-zeroAtoms     4.0
Name: ranked, dtype: float64

In [128]:
## We can save the Frame object to a pickle file and read it later or in another jupyter notebook.
Cu_slabs.to_pickle('Cu_slabs_G')

In [129]:
Cu_slabs.pivot_table(index=['surface_ref', 'adsorbate'], columns='ranked', values='ads_E')

ranked                      1.0        4.0
surface_ref adsorbate                     
100-2x2x4   CO        -0.598439        NaN
            H          0.124054        NaN
            O          0.474743        NaN
            OH        -0.014120  14.738787

# Multiindex Frame containing single atom information

In [130]:
# We can store information for individual atoms, for example chemical element, Bader charge or magnetic moment in a Multiindex DataFrame.
# The Name of the calculation and the index of every atom are both indexes that point to an information.  
Cu_slabs_atomsindex = Atommultiindex(Cu_slabs, struc_file='CONTCAR')
Cu_slabs_atomsindex

Symbols
100-2x2x4              0       Cu
                       1       Cu
                       2       Cu
                       3       Cu
                       4       Cu
...                           ...
100-2x2x4-OH-missmatch 11      Cu
                       12      Cu
                       13      Cu
                       14      Cu
                       15      Cu

[120 rows x 1 columns]